# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

In [ ]:
#selenium - ferramenta de automação web
#!pip install selenium


In [27]:
# Passo 1 - pegar a cotação do dolar
    #Abrir o navegador
    #Abrir o google
    #Pesquisar cotação
    #Pegar cotação
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
navegador = wd.Chrome()

def clear(xpath):
    navegador.find_element('xpath', xpath).clear()
def limpar_google():
    navegador.find_element('xpath', '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').clear()
def cotar_moeda(moeda):
    navegador.find_element('xpath', '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys(moeda)
    navegador.find_element('xpath', '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys(Keys.ENTER)
def click(link):
    navegador.find_element('xpath', link).click()

navegador.get('https://www.google.com/search?q=google&source=hp&ei=qE_jYqHkLreF5OUP_4SjmAM&iflsig=AJiK0e8AAAAAYuNduAW92RCwoLiMEt-KMUntz8oAlHLb&ved=0ahUKEwih2IPyj535AhW3ArkGHX_CCDMQ4dUDCAc&uact=5&oq=google&gs_lcp=Cgdnd3Mtd2l6EAMyCgguEMcBENEDEEMyBAgAEEMyBQgAEJECMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDMgQIABBDOgUIABCABDoOCC4QgAQQxwEQ0QMQ1AI6BwguENQCEENQAFi6E2CZFWgAcAB4AIABiwGIAcEFkgEDMC42mAEAoAEB&sclient=gws-wiz')
limpar_google()
cotar_moeda('cotação dolar')
dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
limpar_google()
cotar_moeda('cotação euro')
euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
limpar_google()
cotar_moeda('cotação ouro')
click('//*[@id="rso"]/div[3]/div/div[1]/div/a/h3')
ouro = navegador.find_element('xpath', '//*[@id="nacional"]').get_attribute('value')

ouro = ouro.replace(',', '.')
navegador.quit()
# Passo 4 - atualizar a base de dados
# Passo 5 - Atualizar base de dados
# Passo 6 - Exportar a base de dados atualizada

5.1845 5.288399999999999 294.09


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [81]:
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [82]:
tabela.loc[tabela['Moeda']=='Dólar', 'Cotação'] = float(dolar)
tabela.loc[tabela['Moeda']=='Euro', 'Cotação'] = float(euro)
tabela.loc[tabela['Moeda']=='Ouro', 'Cotação'] = float(ouro)
#Preço de compra = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
#Preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']
tabela['Preço de Venda'] = tabela['Preço de Venda'].map('R${:.2f}'.format)
tabela['Preço de Compra'] = tabela['Preço de Compra'].map('R${:.2f}'.format)
tabela['Preço Original'] = tabela['Preço Original'].map('R${:.2f}'.format)
tabela['Cotação'] = tabela['Cotação'].map('R${:.2f}'.format)
tabela['Margem'] = tabela['Margem'].map('{:.1%}'.format)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,R$999.99,Dólar,R$5.18,R$5184.45,140.0%,R$7258.23
1,Carro Renault,R$4500.00,Euro,R$5.29,R$23797.80,200.0%,R$47595.60
2,Notebook Dell,R$899.99,Dólar,R$5.18,R$4666.00,170.0%,R$7932.20
3,IPhone,R$799.00,Dólar,R$5.18,R$4142.42,170.0%,R$7042.11
4,Carro Fiat,R$3000.00,Euro,R$5.29,R$15865.20,190.0%,R$30143.88
5,Celular Xiaomi,R$480.48,Dólar,R$5.18,R$2491.05,200.0%,R$4982.10
6,Joia 20g,R$20.00,Ouro,R$294.09,R$5881.80,115.0%,R$6764.07


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
tabela.to_excel('Produtos Novo.xlsx', index=False)